In [1]:
!pip install streamlit
!npm install -g localtunnel
!pip install streamlit requests pandas plotly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸Requirement already satisfied: streamlit in /usr/local/lib/python3.11/dist-packages (1.42.2)


In [52]:
%%writefile app_ui.py
import streamlit as st
import requests
import pandas as pd
import tempfile
import os

# ✅ Automatically fetch ngrok/Cloudflare Tunnel URL (Modify this for production)
BACKEND_URL = os.getenv("BACKEND_URL", "https://caf9-34-147-23-11.ngrok-free.app")  # Update with latest ngrok URL

# ✅ API Endpoints
ANALYZE_API_URL = f"{BACKEND_URL}/analyze_legal_document"
CHATBOT_API_URL = f"{BACKEND_URL}/chatbot"
VIDEO_ANALYZE_API_URL = f"{BACKEND_URL}/analyze_video"
RISK_GRAPH_API_URL = f"{BACKEND_URL}/download_risk_chart"  # ✅ Corrected endpoint for risk graph

# ✅ Configure Streamlit
st.set_page_config(page_title="AI-Powered Legal Assistant", layout="wide")

# ✅ Sidebar Navigation
menu = st.sidebar.radio("📌 Navigation", ["🏠 Home", "📊 Legal Document Analyzer", "🎥 Video Analyzer", "🤖 Legal Chatbot"])

if menu == "🏠 Home":
    st.title("📜 AI-Powered Legal & Video Assistant")
    st.markdown("👋 Welcome to your AI-powered legal and video assistant! Choose an option from the sidebar.")

elif menu == "📊 Legal Document Analyzer":
    st.title("📑 Legal Document Analyzer")
    uploaded_file = st.file_uploader("📂 Upload a PDF document", type=["pdf"])

    if uploaded_file:
        st.success("✅ File uploaded successfully! Analyzing...")

        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
            temp_file.write(uploaded_file.getbuffer())
            temp_file_path = temp_file.name

        files = {"file": open(temp_file_path, "rb")}
        try:
            with st.spinner("⏳ Processing your document... Please wait..."):
                response = requests.post(ANALYZE_API_URL, files=files, timeout=120)
                response.raise_for_status()
                result = response.json()

            # ✅ Fix: Ensure Summary Always Displays
            st.subheader("📑 Document Summary")
            summary_text = result.get("summary", "⚠️ No summary available for this document.")
            st.write(summary_text)

            # ✅ Display Risk Assessment with Image
            st.subheader("⚠️ Legal Risk Assessment")
            risk_scores = result.get("risk_scores", None)

            if risk_scores:
                try:
                    # ✅ Load and display risk chart properly
                    response = requests.get(RISK_GRAPH_API_URL)
                    if response.status_code == 200:
                        st.image(response.content, caption="📊 Risk Assessment Overview")
                    else:
                        st.warning("⚠️ Risk chart could not be loaded.")

                    st.json(risk_scores)  # Show risk scores in a structured format
                except requests.exceptions.RequestException:
                    st.warning("⚠️ Risk assessment data not available.")
            else:
                st.warning("⚠️ No risk assessment data available.")

        except requests.exceptions.RequestException as e:
            st.error(f"❌ API Error: {e}")
        finally:
            os.remove(temp_file_path)  # Cleanup temporary file

elif menu == "🎥 Video Analyzer":
    st.title("🎬 Video Content Analyzer")
    uploaded_video = st.file_uploader("📂 Upload a Video File", type=["mp4", "avi", "mov", "mpeg4"])

    if uploaded_video:
        st.success("✅ Video uploaded successfully! Analyzing...")

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
            temp_file.write(uploaded_video.getbuffer())
            temp_file_path = temp_file.name

        files = {"file": open(temp_file_path, "rb")}
        try:
            with st.spinner("⏳ Processing your video... Please wait..."):
                response = requests.post(VIDEO_ANALYZE_API_URL, files=files, timeout=180)
                response.raise_for_status()
                result = response.json()

            # ✅ Fix: Handle Cases Where No Transcript is Found
            st.subheader("📜 Video Transcript")
            if result.get("status") == "success":
                transcript = result.get("transcript", "No transcript available.")
                st.write(transcript)
            else:
                error_message = result.get("message", "⚠️ No valid data returned from the video analysis API.")
                st.warning(error_message)

        except requests.exceptions.RequestException as e:
            st.error(f"❌ API Error: {e}")
        finally:
            os.remove(temp_file_path)  # Cleanup temporary file

elif menu == "🤖 Legal Chatbot":
    st.title("🤖 AI Legal Chatbot")
    st.write("💡 Ask any legal question and get AI-powered responses!")
    user_query = st.text_input("💬 Ask a legal question:")

    if st.button("🔍 Get Answer"):
        if user_query.strip():
            with st.spinner("⏳ Thinking... Please wait..."):
                try:
                    response = requests.post(CHATBOT_API_URL, json={"query": user_query}, timeout=60)
                    response.raise_for_status()
                    chatbot_response = response.json().get("answer", "No response available.")

                    # ✅ Fix: Display Chatbot Answer Properly
                    st.subheader("💡 AI's Response")
                    st.success(chatbot_response)

                except requests.exceptions.RequestException as e:
                    st.error(f"❌ API Error: {e}")
        else:
            st.warning("⚠️ Please enter a question before submitting.")


Overwriting app_ui.py


In [53]:
!cloudflared tunnel run

"cloudflared tunnel run" requires the ID or name of the tunnel to run as the last command line argument or in the configuration file.
See 'cloudflared tunnel run --help'.


In [54]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/


In [55]:
!pkill -9 streamlit


In [ ]:



# ✅ Run Streamlit and Expose it via Cloudflare Tunnel
!streamlit run app_ui.py & sleep 5 && cloudflared tunnel --url http://localhost:8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.81.56.226:8501

2025-02-25T12:12:08Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-02-25T12:12:08Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-02-25T12:12:11Z INF +--------------------------------------------------------------------------------------------+
2025-02-25T12:12:11Z INF |